# [Module 7.1] Similar SIMS 추천 결과 분석

이 노트북을 실행하기 전에 먼저 "데이타 셋의 분리"에 대해서 다시 한번 보겠습니다.
Training Data (학습 데이터 셋)는 Solution Version을 학습하여 Campaign을 생성하였고 Validation(holdout, 검증 데이터 셋)은 이 노트북에서 Campaign을 통해 나온 추천 결과와 비교하여 성능 지표를 계산 합니다. 
- Coldstart는 현재 노트북에서 사용하지 않으므로 무시하시기 바랍니다.
<p>
<div>
<img src="static/imgs/img_datasplit50_v1.png", width="800">
</div>



이 노트북에서는 아래와 같은 작업을 합니다.

- 유저 개인별 아이템 클릭한 아이템의 추천 분석
    - **캠페인을 통해서 추천 결과가 나오면, 어떻게 해서 이런 추천 결과가 나올까 라는 호기심이 생깁니다.** 이런 호기심을 충족하기 위해서, (1) 학습 데이터 셋에서의 최근 인터랙션의 마지막 영화를 확인하고, (2) 검증 데이터 셋의 리스트를 확인 합니다. 이후 (3) 캠페인을 통한 추천 결과 리스트를 확인 합니다. (3)의 추천 결과는 (1) 의 기반 위에서 나온 겻을 확인할 수 있고, (3) 과 (2)를 비교하여 추천 정확도를 확인 합니다.


- 캠페인을 통한 Validation(검증) 데이터 셋의 지표 분석 (전체 6040명) 
    - **실제로 캠페인을 통한 추천의 성능이 얼마나 잘 되었는지를 확인 합니다.**




---
이 노트북은 실행 시간이 약 3분 소요 됩니다. 하지만 분석된 결과를 보시려면 약 20분 정도 걸립니다.


# 1. 환경 셋업

In [1]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import os
import boto3
from metrics import ndcg_at_k, precision_at_k, mean_reciprocal_rank

from tqdm import notebook
from IPython.display import display, HTML
from IPython.display import display as dp

In [2]:
%store -r

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


# 2. 기본 데이터 셋 가져오기 

다음과 같은 데이터 셋을 가져 옵니다.
- 학습에 사용된 데이터 셋
- 검증에 사용할 데이터 셋

In [4]:
from utils import get_rich_dataset

# 학습에 사용된 warm_train 데이터 셋 로딩
df_warm_train = pd.read_csv(warm_train_interaction_filename)
# Validation 인 holdout 데이터 셋 로딩
df_holdout = pd.read_csv(validation_interaction_filename)
# item 정보 로딩
item_meta = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
# 영황 타이틀, 장르를 포함한 정보 리턴
df_warm_train_rich = get_rich_dataset(df_warm_train, item_meta)
df_warm_train_rich = df_warm_train_rich.sort_values('TIMESTAMP').copy()
# 영황 타이틀, 장르를 포함한 정보 리턴
df_holdout_rich = get_rich_dataset(df_holdout, item_meta)
df_holdout_rich = df_holdout_rich.sort_values('TIMESTAMP').copy()

num_history = 5 # warm_train에서 최근 데이타 보여줄 수
num_recommend = 10 # 캠페인을 통한 추천에서 보여줄 수


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


# 3. 유저별 "유사 아이템 추천" 분석

In [5]:
def track_item_recommend(user_id, campaign_arn):
    '''
    유저의 선택한 영화 부터 추천에 대한 리스트 제공
    '''
    # show the last interaction (warm_train) out of training data
    rec_items_movies, last_item_df, last_item_id = get_similar_items_richdata(user_id, df_warm_train_rich, campaign_arn, item_meta, num_history, num_recommend)

    display(HTML("<font><b>The last item the user clicked: </b></font>"))
    display(last_item_df)

    # show validation data
    display(HTML("<font><b>The validation list (Holdout Dataset): </b></font>"))
    display(df_holdout_rich[df_holdout_rich.USER_ID == user_id].tail(100))

    # 캠페인에서의 추천 리스트
    display(HTML("<font><b>The recommended list by Campaign: </b></font>"))
    display(rec_items_movies)

    # 매치 리스트: validation (warm_holdout) and 캠페인에서의 추천 리스트
    relevance_list = get_item_relevance_list(campaign_arn, df_holdout, user_id, last_item_id)
    display(HTML("<font><b>The matched list between validation (Holdout Dataset)and recommendation: </b></font>"))
    display(relevance_list)

def get_similar_items_richdata(user_id, df_rich, campaign_arn, item_meta, num_history, num_recommend):
    '''
    학습된 데이터(Warm_Train)에서 최신 리스트 및 추천 아이템을 제공 함
    '''
    history_items = df_rich[df_rich['USER_ID']==user_id].tail(num_history)
    last_item_df = history_items[-1:]
#     print("##################")
    last_item_id = last_item_df["ITEM_ID"].values[0]
    
    
    rec_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                itemId = str(last_item_id)
            )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items_movies = item_meta.set_index('ITEM_ID').loc[rec_items[:num_recommend]]

    return rec_items_movies, last_item_df, last_item_id

def get_item_relevance_list(campaign_arn, df_holdout, user_id, item_id):
    '''
    한명의 유저에 대해서 validation(warm_holdout)와 추천 리스트에서 매치된 것을 제공
    '''
    relevance = []

    true_items = set(df_holdout[df_holdout['USER_ID']==user_id]['ITEM_ID'].values)
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        itemId = str(item_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items) for x in rec_items])
    return relevance

        

## 유저 1
아래의 그림은 유저1 에 대해서, 분석 시나리오 입니다.
- (1) The last item the user clicked
    - 해당 유저의 모델 훈련(솔루선 버전)에 사용된 인터렉션 데이터 중에 가장 최근의 데이터를 보여 줌
        - 이 유저는 Tarzan 을 마지막으로 인터렉션을 함.
- (2) The validation list (Holdout Dataset):
    - 해당 유저에 대해서 모델 훈련에 들어간 이후 부터의 데이터 ("검증데이터", "홀드아웃" 데이터 셋으로 불리움) 를 보여 줌.
        - 노트르담, 벅스 라이프, 뮬란 의 영화가 있음.            
- (3) The recommended list by Campaign:
    - 타잔 (item_id == 2687) 를 제공하여 similar_sims 레서피로 만든 솔루션 버전을 서빙하는 캠페인으로 부터 추천 결과 임.
- (4) The matched list between validation (Holdout Dataset)and recommendation:
    - (2), (3) 을 일차여 부를 0, 1 로 구분 함. 1이 "일치" 를 의미

![story_similar_item.png](img/story_similar_item.png)

In [6]:
user_id = 1    
track_item_recommend(user_id,  sims_campaign_arn)    


,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
11384,1,2687,Tarzan (1999),Animation|Children's,978824268,2001-01-06 23:37:48


,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
166,1,783,"Hunchback of Notre Dame, The (1996)",Animation|Children's|Musical,978824291,2001-01-06 23:38:11
0,1,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,978824291,2001-01-06 23:38:11
101,1,1907,Mulan (1998),Animation|Children's,978824330,2001-01-06 23:38:50


,TITLE,GENRE
ITEM_ID,,
2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
1907,Mulan (1998),Animation|Children's
2085,101 Dalmatians (1961),Animation|Children's
2137,Charlotte's Web (1973),Animation|Children's
783,"Hunchback of Notre Dame, The (1996)",Animation|Children's|Musical
1,Toy Story (1995),Animation|Children's|Comedy
594,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
2762,"Sixth Sense, The (1999)",Thriller
2090,"Rescuers, The (1977)",Animation|Children's


[[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

## 유저 2


In [7]:
user_id = 2
track_item_recommend(user_id,  sims_campaign_arn)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
47355,2,1690,Alien: Resurrection (1997),Action|Horror|Sci-Fi,978300051,2000-12-31 22:00:51


,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
469,2,3257,"Bodyguard, The (1992)",Action|Drama|Romance|Thriller,978300073,2000-12-31 22:01:13
555,2,736,Twister (1996),Action|Adventure|Romance|Thriller,978300100,2000-12-31 22:01:40
268,2,292,Outbreak (1995),Action|Drama|Thriller,978300123,2000-12-31 22:02:03
675,2,95,Broken Arrow (1996),Action|Thriller,978300143,2000-12-31 22:02:23
215,2,1687,"Jackal, The (1997)",Action|Thriller,978300174,2000-12-31 22:02:54
734,2,1917,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,978300174,2000-12-31 22:02:54
339,2,1544,"Lost World: Jurassic Park, The (1997)",Action|Adventure|Sci-Fi|Thriller,978300174,2000-12-31 22:02:54


,TITLE,GENRE
ITEM_ID,,
1917,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller
196,Species (1995),Horror|Sci-Fi
1544,"Lost World: Jurassic Park, The (1997)",Action|Adventure|Sci-Fi|Thriller
1371,Star Trek: The Motion Picture (1979),Action|Adventure|Sci-Fi
2393,Star Trek: Insurrection (1998),Action|Sci-Fi
1779,Sphere (1998),Adventure|Sci-Fi|Thriller
780,Independence Day (ID4) (1996),Action|Sci-Fi|War
3701,Alien Nation (1988),Crime|Drama|Sci-Fi
1037,"Lawnmower Man, The (1992)",Action|Sci-Fi|Thriller


[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]

## 유저 5002


In [8]:
user_id = 5002
track_item_recommend(user_id,  sims_campaign_arn)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
8071,5002,595,Beauty and the Beast (1991),Animation|Children's|Musical,963027451,2000-07-08 03:37:31


,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
31308,5002,2160,Rosemary's Baby (1968),Horror|Thriller,963028123,2000-07-08 03:48:43
18429,5002,2791,Airplane! (1980),Comedy,963278308,2000-07-11 01:18:28
43474,5002,2416,Back to School (1986),Comedy,963278347,2000-07-11 01:19:07
29575,5002,674,Barbarella (1968),Adventure|Sci-Fi,963278381,2000-07-11 01:19:41
6893,5002,3255,"League of Their Own, A (1992)",Comedy|Drama,963281233,2000-07-11 02:07:13
38931,5002,3100,"River Runs Through It, A (1992)",Drama,963281305,2000-07-11 02:08:25
35582,5002,1029,Dumbo (1941),Animation|Children's|Musical,963281378,2000-07-11 02:09:38
12951,5002,708,"Truth About Cats & Dogs, The (1996)",Comedy|Romance,963281736,2000-07-11 02:15:36
22706,5002,1441,Benny & Joon (1993),Comedy|Romance,963281749,2000-07-11 02:15:49
31066,5002,383,Wyatt Earp (1994),Western,963282010,2000-07-11 02:20:10


,TITLE,GENRE
ITEM_ID,,
364,"Lion King, The (1994)",Animation|Children's|Musical
594,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
1022,Cinderella (1950),Animation|Children's|Musical
1029,Dumbo (1941),Animation|Children's|Musical
1282,Fantasia (1940),Animation|Children's|Musical
2687,Tarzan (1999),Animation|Children's
1,Toy Story (1995),Animation|Children's|Comedy
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2080,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance


[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]

# 4. Validaton(holdout, 검증) 데이터 세트 평가하기

이번 파트에서는 앞장에 남겨두었던 데이터 세트를 활용하여 모델 성능을 평가 하도록 합니다.
테스트 데이터 셋에 있는 모든 고유한 사용자에 대해 테스트 데이터 세트 Interaction Ground Truth data와 Campaign에서 생성된 결과를 비교 하도록 합니다.

#### Help 함수

In [9]:

def evaluate_relevance(relevance):
    '''
    relevance 입력 받으면, 지표를 사전으로 리턴 함
    '''
    mrr = np.mean([mean_reciprocal_rank(r) for r in relevance])
    p_at_5= np.mean([precision_at_k(r, 5) for r in relevance])
    p_at_10 = np.mean([precision_at_k(r, 10) for r in relevance])
    p_at_25 = np.mean([precision_at_k(r, 25) for r in relevance])
    ndcg_at_5 = np.mean([ndcg_at_k(r, 5) for r in relevance])
    ndcg_at_10 = np.mean([ndcg_at_k(r, 10) for r in relevance])
    ndcg_at_25 = np.mean([ndcg_at_k(r, 25) for r in relevance])    
#     print('mean_reciprocal_rank: ',mrr)
#     print('precision_at_5: ',p_at_5)
#     print('precision_at_10: ',p_at_10)
#     print('precision_at_25: ', p_at_25)
#     print('normalized_discounted_cumulative_gain_at_5: ', ndcg_at_5)
#     print('normalized_discounted_cu{}mulative_gain_at_10: ',ndcg_at_10 )
#     print('normalized_discounted_cumulative_gain_at_25: ',ndcg_at_25 )    

    metric_dict = {}
    metric_dict['mrr'] = round(mrr,3)
    metric_dict['ndcg_at_5'] = round(ndcg_at_5,3)        
    metric_dict['ndcg_at_10'] = round(ndcg_at_10,3)            
    metric_dict['ndcg_at_25'] = round(ndcg_at_25,3)                
    metric_dict['p_at_5'] = round(p_at_5,3)    
    metric_dict['p_at_10'] = round(p_at_10, 3)
    metric_dict['p_at_25'] = round(p_at_25,3)    
        
    return metric_dict

def build_metric_matrix(solution,metric_dict):
    metrics.append([solution,
                        metric_dict['mrr'],
                        metric_dict['p_at_5'],
                        metric_dict['p_at_10'],
                        metric_dict['p_at_25'],
                        metric_dict['ndcg_at_5'],
                        metric_dict['ndcg_at_10'],
                        metric_dict['ndcg_at_25']

])


In [10]:
def get_relevance_item_list(campaign_arn, df_rich, df_holdout, test_user_list):
    '''
    유저 그룹에 대해서 validation(warm_holdout)와 추천 리스트에서 매치된 것을 제공    
    '''
    relevance = []
    for user_id in test_user_list:
        true_items = set(df_holdout[df_holdout['USER_ID']==user_id]['ITEM_ID'].values)

        # Get user history
        history_items = df_rich[df_rich['USER_ID']==user_id].tail(2)
#         print("history_items df: \n")        
#         dp(history_items) 
        # Get last item id
        last_item_df = history_items[-1:]
        last_item_id = last_item_df["ITEM_ID"].values[0]
        # print("last_item_id: ", last_item_id)

        rec_response = personalize_runtime.get_recommendations(
            campaignArn = campaign_arn,
            itemId = str(last_item_id)
        )
        rec_items = [int(x['itemId']) for x in rec_response['itemList']]
        relevance.append([int(x in true_items) for x in rec_items])
    return relevance


## 4.1. 샘플 Validation 결과 확인 하기

아래 과정은 num_test_user = 6040 (전체) 으로 하면 약 3분 소요 됩니다.
디폴트로 1을 설정 합니다.
- user_Id == 1 에 해당 됨

In [19]:
test_users = df_holdout['USER_ID'].unique()

# num_validation_test_user = 6040
num_validation_test_user = 1
test_user_list = test_users[:num_validation_test_user]
print("user_id: ", test_user_list[0])

user_id:  1


In [16]:
%%time

metrics=[] # 변수 선언
# user-perssonalization
relevance = get_relevance_item_list(sims_campaign_arn, df_warm_train_rich, df_holdout, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("similar-sims",metrics_eval_rel)


CPU times: user 14.6 ms, sys: 29 µs, total: 14.6 ms
Wall time: 111 ms


In [17]:
val_metrics = pd.DataFrame(metrics, 
                           columns=['recipe','mrr','ncdg@5','ncdg@10','ncdg@25','p@5','p@10','p@25'])
val_metrics

,recipe,mrr,ncdg@5,ncdg@10,ncdg@25,p@5,p@10,p@25
0,similar-sims,0.5,0.4,0.2,0.12,0.544,0.544,0.647


## 4.2. 전체 Validation(검증) 데이터 셋으로 지표 분석




Validation 데이터 셋 (6040명)과 Campaign의 추천 결과를 비교하여 만든 평가 지표



In [14]:
%%time

# num_validation_test_user = 6040
num_validation_test_user = 6040
test_user_list = test_users[:num_validation_test_user]

metrics=[] # 변수 선언
# user-perssonalization
relevance = get_relevance_item_list(sims_campaign_arn, df_warm_train_rich, df_holdout, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("similar-sims",metrics_eval_rel)

val_metrics = pd.DataFrame(metrics, 
                           columns=['recipe','mrr','ncdg@5','ncdg@10','ncdg@25','p@5','p@10','p@25'])
val_metrics

CPU times: user 21.5 s, sys: 1.79 s, total: 23.3 s
Wall time: 2min 41s


,recipe,mrr,ncdg@5,ncdg@10,ncdg@25,p@5,p@10,p@25
0,similar-sims,0.217,0.084,0.069,0.051,0.172,0.221,0.299
